# Homework 2 - Wikipedia Web Traffic Time Series

У вас есть данные по посещению 1000 страниц  Википедии из разных стран и разных девайсов ( \*  * данные взяты из [Kaggle соревнования](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train* и *wikipedia_test* - содержат данные о трафике. Это файлы csv, где каждая строка соответствует определенной статье, и каждый столбец соответствует конкретной дате. В некоторых записях отсутствуют данные. Названия страниц содержат проект Википедии (например, en.wikipedia.org), тип доступа (например, desktop) и тип агента (например, spider). Другими словами, каждое имя статьи имеет следующий формат: «name_project_access_agent» (например, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам нужно ответить на [вопросы](https://docs.google.com/forms/d/e/1FAIpQLScQYrgg50C1hJeeNQflumKqJ4rWihDj9Q9HWMy-ls7qt3iAJA/viewform?usp=sf_link) и попробовать сделать самую простую модель которая сможет предсказывать будущие посещения. 

Вот примеры временных рядов посещаемости страниц Википедии (*синие* - обучающая выборка, *зеленые* - предсказания модели победителя соревнования на Kaggle, *оранжевые* - реальные значения):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("./data/wikipedia_train.csv")
test = pd.read_csv("./data/wikipedia_test.csv")

## Data Analysis

In [3]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

In [19]:
train['country'] = train['Page'].apply(get_language)
rus_len = len(train[train['country'] == 'ru'])
print(f'{rus_len} страниц из русской Википедии в датасете')


102 страниц из русской Википедии в датасете


In [57]:
pd.set_option('display.max_rows', 102)
train_rus = train[train['country'] == 'ru']
train_rus["sum"]  = train_rus.iloc[:, 1:-1].sum(axis=1)
train_rus

/tmp/ipykernel_592511/3442180901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_rus["sum"]  = train_rus.iloc[:, 1:-1].sum(axis=1)


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31,country,sum
6,22_апреля_ru.wikipedia.org_all-access_spider,18.0,13.0,29.0,28.0,25.0,5.0,21.0,24.0,35.0,...,16.0,28.0,12.0,6.0,13.0,7.0,19.0,6.0,ru,6050.0
8,6-я_армия_(Третий_рейх)_ru.wikipedia.org_deskt...,16.0,43.0,28.0,33.0,43.0,34.0,21.0,57.0,21.0,...,39.0,69.0,66.0,34.0,47.0,53.0,40.0,51.0,ru,62389.0
38,Apple_ru.wikipedia.org_mobile-web_all-agents,391.0,490.0,436.0,510.0,442.0,440.0,440.0,418.0,414.0,...,508.0,470.0,481.0,522.0,565.0,493.0,701.0,662.0,ru,278139.0
154,Dishonored_ru.wikipedia.org_all-access_all-agents,792.0,844.0,838.0,812.0,1204.0,945.0,786.0,747.0,722.0,...,810.0,816.0,791.0,701.0,643.0,792.0,758.0,620.0,ru,303653.0
180,Facebook_ru.wikipedia.org_desktop_all-agents,1665.0,1419.0,1440.0,1062.0,1169.0,1496.0,1509.0,1602.0,1383.0,...,1786.0,1817.0,1927.0,1401.0,1318.0,1708.0,1748.0,2235.0,ru,929291.0
548,Sukhoi_Superjet_100_ru.wikipedia.org_desktop_a...,785.0,762.0,619.0,467.0,561.0,877.0,949.0,762.0,710.0,...,656.0,650.0,732.0,919.0,681.0,791.0,744.0,879.0,ru,442688.0
602,Victoria’s_Secret_ru.wikipedia.org_all-access_...,4.0,5.0,7.0,10.0,13.0,4.0,4.0,6.0,6.0,...,6.0,5.0,12.0,7.0,6.0,11.0,11.0,15.0,ru,8471.0
639,"А_зори_здесь_тихие_(фильм,_1972)_ru.wikipedia....",147.0,126.0,158.0,208.0,239.0,143.0,175.0,178.0,156.0,...,187.0,142.0,111.0,136.0,196.0,97.0,104.0,99.0,ru,120824.0
640,Агенты_«Щ.И.Т.»_ru.wikipedia.org_desktop_all-a...,507.0,471.0,520.0,458.0,540.0,534.0,577.0,556.0,551.0,...,681.0,699.0,655.0,563.0,713.0,731.0,703.0,749.0,ru,452597.0
641,Атлетико_Мадрид_ru.wikipedia.org_mobile-web_al...,1049.0,892.0,417.0,316.0,301.0,401.0,429.0,355.0,354.0,...,397.0,841.0,676.0,526.0,427.0,380.0,334.0,350.0,ru,236691.0


## Forecasting

Нужно преобразовать `train` данные в следующий формат:

In [6]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-08-22,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31
0,15._November_de.wikipedia.org_desktop_all-agents,32.0,26.0,22.0,22.0,29.0,49.0,20.0,27.0,19.0,...,29.0,23.0,31.0,25.0,27.0,23.0,17.0,26.0,23.0,37.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2.0,3.0,5.0,3.0,5.0,3.0,7.0,8.0,7.0,...,5.0,5.0,6.0,5.0,4.0,11.0,2.0,0.0,7.0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,1.0,3.0,2.0,2.0,1.0,10.0,2.0,1.0,4.0,...,4.0,3.0,2.0,3.0,2.0,4.0,2.0,0.0,5.0,4.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,3.0,3.0,3.0,8.0,12.0,12.0,8.0,12.0,23.0,...,10.0,14.0,26.0,5.0,29.0,23.0,17.0,16.0,12.0,14.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,24.0,40.0,23.0,49.0,88.0,25.0,31.0,76.0,51.0,...,134.0,162.0,208.0,179.0,108.0,99.0,49.0,80.0,113.0,173.0


Таким образом у вас каждая сточка содержит набор фич (`Page`, `date`) и целевую переменную (`Visits`). Преобразовать данные в такой формат поможет функция `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

Оценивать качество предсказаний мы будем с помощью [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [7]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

Что б оценить прогноз нужно представить `test` данные в следующем формате:

In [9]:
prediction.head()

NameError: name 'prediction' is not defined

И затем вызвать функцию для подсчета метрики:

In [ ]:
pandas_smape(prediction)

### Last day baseline

Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test)

### Median baseline

Нужно сделать прогноз на основе медианы за последние **30** дней из `train`. 

А затем улучшить предсказания используя информацию выходной это или нет (воспользуйтесь функцией [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) и разные окна для подсчета медианы (7 дней, 60 дней и тд)

Вам поможет функция `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)